In [7]:
import pandas as pd

# Data Cleaning

In [8]:
df_historical_data = pd.read_csv('fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('fifa_worldcup_fixture.csv')
df_missing_data = pd.read_csv('fifa_worldcup_missing_data.csv')

## Cleaning df_fixture

In [9]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

## Cleaning df_missing_data and adding it to df_historical_data

In [10]:
# df_missing_data[df_missing_data['home'].isnull()]
df_missing_data.dropna(inplace=True)
df_historical_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year', inplace=True)
df_historical_data

,home,score,away,year
0,France,0–1,Senegal,2002
34,Sweden,1–1,Argentina,2002
35,Nigeria,0–0,England,2002
36,Croatia,0–1,Mexico,2002
37,Italy,2–0,Ecuador,2002
...,...,...,...,...
282,Japan,0–1,Costa Rica,2022
283,Spain,1–1,Germany,2022
284,Japan,2–1,Spain,2022
286,Morocco,0–0,Croatia,2022


## Cleaning df_historical_data

In [11]:
# deleting match with walk over
delete_index = df_historical_data[df_historical_data['home'].str.contains('Sweden') &
                                  df_historical_data['away'].str.contains('Austria')].index

df_historical_data.drop(index=delete_index, inplace=True)

# cleanning score and home/away columns
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)
df_historical_data['home'] = df_historical_data['home'].str.strip() # clean blank spaces: Yugoslavia twice
df_historical_data['away'] = df_historical_data['away'].str.strip()

# splitting score columns into home and away goals and dropping score column
df_historical_data[['HomeGoals', 'AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data.drop('score', axis=1, inplace=True)

# renaming columns and changing data types
df_historical_data.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam', 
                                   'year':'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

# creating new column "totalgoals"
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Senegal,2002,0,1,1
34,Sweden,Argentina,2002,1,1,2
35,Nigeria,England,2002,0,0,0
36,Croatia,Mexico,2002,0,1,1
37,Italy,Ecuador,2002,2,0,2
...,...,...,...,...,...,...
282,Japan,Costa Rica,2022,0,1,1
283,Spain,Germany,2022,1,1,2
284,Japan,Spain,2022,2,1,3
286,Morocco,Croatia,2022,0,0,0


# Exporting clean dataframes

In [12]:
df_historical_data.to_csv('clean_fifa_worldcup_matches.csv',index=False)
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv',index=False)

# Extra verifications

In [13]:
# verify number of matches per competition
years = [2002, 2006, 2014, 2022]

for year in years:
    print(year, len(df_historical_data[df_historical_data['Year']==year]))

2002 64
2006 64
2014 64
2022 64


In [14]:
# verify data collected for a team
print(df_historical_data[df_historical_data['HomeTeam'].str.contains('Turkey')])
print(df_historical_data[df_historical_data['AwayTeam'].str.contains('Turkey')])

   HomeTeam AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
17   Turkey    China  2002          3          0           3
       HomeTeam AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
54        Japan   Turkey  2002          0          1           1
59      Senegal   Turkey  2002          0          1           1
61       Brazil   Turkey  2002          1          0           1
62  South Korea   Turkey  2002          2          3           5
12       Brazil   Turkey  2002          2          1           3
15   Costa Rica   Turkey  2002          1          1           2
